#### Simple implementation of LSTM in Tensorflow
Ref: http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/


In [ ]:
#Imports
import numpy as np
import random
from random import shuffle
import tensorflow as tf
import os

# from tensorflow.models.rnn import rnn_cell
# from tensorflow.models.rnn import rnn

#### Loading data

In [ ]:
# Loading data 
NUM_EXAMPLES = 10000

train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]

print ("test and training data loaded")

test and training data loaded


In [ ]:
# Definining values
LOG_DIR='./logdir'
custom_global_step=0
batch_size = 1000
no_of_batches = int(len(train_input) / batch_size)
epoch = 50
num_hidden = 24

#### Defining Model

In [ ]:
data = tf.placeholder(tf.float32, [None, 20,1]) #Number of examples, number of input, dimension of each input
target = tf.placeholder(tf.float32, [None, 21])

In [ ]:
cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

# Defining variables for writing summary
tf.summary.scalar("cross_entropy", cross_entropy)
summary_op = tf.summary.merge_all()

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


#### Running session

In [ ]:
init_op = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init_op)

saver = tf.train.Saver()
train_writer = tf.summary.FileWriter('./logdir', sess.graph)

for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        _,summ_string=sess.run([minimize,summary_op],feed_dict={data: inp, target: out})
        train_writer.add_summary(summ_string,custom_global_step) # Writing summary to disc
        custom_global_step+=1
    
    print ('Epoch {}'.format(str(i)))
incorrect = sess.run(error,{data: test_input, target: test_output})
print (sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
print('Error {:3.1f}%'.format(100 * incorrect))
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), global_step=custom_global_step+1) 


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
[[  4.16957409e-05   1.51184679e-04   1.01161662e-04   9.58907822e-06
    4.05898754e-05   1.91918862e-05   1.06413499e-04   1.19468896e-04
    6.81659789e-04   3.68862529e-03   1.01423375e-02   1.20425723e-01
    6.04755163e-01   2.11854488e-01   3.59153561e-02   9.94639937e-03
    1.51711481e-03   1.62242650e-04   2.42255810e-05   3.42401327e-05
    2.63100170e-04]]
Error 22.6%


'./logdir/model.ckpt-501'

#### Tensorboard 

In [ ]:
# Checking the items in the log directory
!tensorboard --inspect --logdir='./logdir/'

Processing event files... (this can take a few minutes)

Found event files in:
./logdir/

These tags are in ./logdir/:
audio -
histograms -
images -
scalars -
tensor -

Event statistics for ./logdir/:
audio -
graph
   first_step           0
   last_step            0
   max_step             0
   min_step             0
   num_steps            1
   outoforder_steps     []
histograms -
images -
scalars -
sessionlog:checkpoint -
sessionlog:start -
sessionlog:stop -
tensor -



In [ ]:
# Running tensorflow
print('If on windows system go to: http://localhost:6006')
!tensorboard --logdir='./logdir'

#print('If on windows system go to: http://localhost:6006')
#!tensorboard --logdir='./logdir’

If on windows system go to: http://localhost:6006
Starting TensorBoard b'47' at http://0.0.0.0:6006
(Press CTRL+C to quit)


In [ ]:
sess.close()